# Imports

In [3]:
pip install mediapipe


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install cv2

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip uninstall tensorflow

^C
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install --upgrade pip

     ---------------------------------------- 2.1/2.1 MB 2.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.2.2
    Uninstalling pip-22.2.2:
      Successfully uninstalled pip-22.2.2
Note: you may need to restart the kernel to use updated packages.


In [1]:
from tensorflow import keras

ImportError: Traceback (most recent call last):
  File "C:\Users\USER\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 18, in swig_import_helper
    fp, pathname, description = imp.find_module('_pywrap_tensorflow', [dirname(__file__)])
  File "C:\Users\USER\anaconda3\lib\imp.py", line 296, in find_module
    raise ImportError(_ERR_MSG.format(name), name=name)
ImportError: No module named '_pywrap_tensorflow'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\USER\anaconda3\lib\site-packages\tensorflow\python\__init__.py", line 66, in <module>
    from tensorflow.python import pywrap_tensorflow
  File "C:\Users\USER\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 28, in <module>
    _pywrap_tensorflow = swig_import_helper()
  File "C:\Users\USER\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 20, in swig_import_helper
    import _pywrap_tens

In [1]:
import cv2
import mediapipe as mp
import time
import numpy as np
import matplotlib.pyplot as plt
from pygame import mixer
from scipy.spatial import distance as dist
#from tensorflow import keras

#opencv objects detection
with open("ClassNames.txt", "r") as fd:
    classNames = fd.read().splitlines()

#Audio Files
TiredAudioFile = "AUDIO/GetSomeRest.wav"
DistratedAudioFile = "AUDIO/StayFocused.wav"
ClosedEyesAudioFile = "AUDIO/WakeUp.wav"
SmokingAudioFile = "AUDIO/StopSmoking.wav"
FoodAudioFile = "AUDIO/StopEating.wav"
PhoneUsageAudioFile = "AUDIO/PutDownPhone.wav"
#classeIdsof objects to detecte
classes= ['1', 30, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 84]

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Functions

### FIRST_CONFIGURATION()
To adapte our model to the different camera positions, we have to save the default position of the driver.

In [3]:
def FIRST_CONFIGURATION(draw=False, camera=0, info=True):
    face_length = 400
    pTime = 0
    cap = cv2.VideoCapture(camera)

    while cv2.waitKey(1) != ord('d') :
        success, img= cap.read()
        if not success:
            print("No camera detected")
            break
        #converting the image from BGR to RGB
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = faceMesh.process(imgRGB)
        faces = []
        if results.multi_face_landmarks:
            for faceLms in results.multi_face_landmarks:
                if draw:
                    mpDraw.draw_landmarks(img,faceLms,mpFaceMesh.FACEMESH_CONTOURS,drawSpec,drawSpec)
                face = []
                for id, lm in enumerate(faceLms.landmark):
                    ih, iw, ic = img.shape
                    x,y = int(lm.x*iw), int(lm.y*ih)
                    #cv2.putText(img, str(id), (x,y),cv2.FONT_HERSHEY_PLAIN,0.8, (0,255,0), 1)
                    face.append([x,y])
                #calculation distances and facing direction
                face_length = dist.euclidean(face[10],face[152])
                left_dist_cfg = dist.euclidean(face[1],face[361])
                right_dist_cfg = dist.euclidean(face[1],face[132])
                direction_cfg= (right_dist_cfg-left_dist_cfg)/face_length
                #afficher la direction [-1,1]
                if info:
                    cv2.putText(img, str(round(direction_cfg,2)), (400,400),cv2.FONT_HERSHEY_PLAIN,2, (0,0,255), 3)
                faces.append(face)
        # fps calcul 
        cTime = time.time()
        fps = 1/(cTime-pTime)
        pTime = cTime 
        cv2.putText(img, "Take your default driving position and press 'd'", (20,70),cv2.FONT_HERSHEY_PLAIN,1.5, (255,100,200), 3)
        cv2.putText(img,f'FPS: {int(fps)}',(70,400),cv2.FONT_HERSHEY_PLAIN, 2, (0,255,0), 2)
        cv2.imshow("First configuration", img)

    cap.release()
    cv2.destroyAllWindows()
    return direction_cfg

### Time_thresh()
To make the alerts smarter and more accurate we used this function to adapte the behavious time threshold to the speed of the vehicle.

In [4]:
def Time_thresh(speed,MinThresh=2,MaxThresh=10,curve=3):
    return(((MaxThresh-MinThresh)/(np.exp(speed/curve)))+MinThresh)

Simulation:

In [5]:
x=np.array(range(101))
Speed = 0
acceleration = True
while cv2.waitKey(10) != 27:
    plt.xlabel("Speed (km/h)")
    plt.ylabel("Time threshold (s)")
    plt.grid('on')
    plt.plot(x,Time_thresh(x,curve=5),'r')
    plt.plot(Speed, Time_thresh(Speed,curve=5), 'ko')
    plt.savefig('ploy.jpg')
    img = cv2.imread('ploy.jpg')
    cv2.putText(img, str(Speed)+'km/h', (200,60),cv2.FONT_HERSHEY_PLAIN,1.5, (0,0,255), 2)
    cv2.putText(img, 'Press ESC to quit', (10,20),cv2.FONT_HERSHEY_PLAIN,1, (0,0,0), 1)
    cv2.imshow("Time Threshold depending on the speed ",img)
    if acceleration:
        Speed += 5
    else:
        Speed -= 5
    if Speed==100:
        acceleration = False
    if Speed==0:
        acceleration = True
    plt.clf()
cv2.destroyAllWindows()

<Figure size 432x288 with 0 Axes>

### Calculations()
Calculates the differente distances of the crucial point to analyse face muscles.

In [6]:
def Calculations(face):
    left_dist = dist.euclidean(face[1],face[361])
    right_dist = dist.euclidean(face[1],face[132])
    down_dist = dist.euclidean(face[1],face[0])
    
    upper_lip=[((face[13][0]+face[312][0]+face[267][0]+face[0][0]+face[37][0]+face[82][0])/6),
               ((face[13][1]+face[312][1]+face[267][1]+face[0][1]+face[37][1]+face[82][1])/6)]
    lower_lip=[((face[14][0]+face[317][0]+face[314][0]+face[17][0]+face[84][0]+face[87][0])/6),
               ((face[14][1]+face[317][1]+face[314][1]+face[17][1]+face[84][1]+face[87][1])/6)]
    
    A_eye_R = dist.euclidean(face[160],face[144])
    B_eye_R = dist.euclidean(face[158],face[153])
    C_eye_R = dist.euclidean(face[33],face[133])
    R_ear = (A_eye_R + B_eye_R) / (2.0 * C_eye_R)

    A_eye_L = dist.euclidean(face[385],face[380])
    B_eye_L = dist.euclidean(face[387],face[373])
    C_eye_L = dist.euclidean(face[362],face[263])
    L_ear = (A_eye_L + B_eye_L) / (2.0 * C_eye_L)
    
    ear = (R_ear + L_ear) / 2.0
    dist1 = dist.euclidean(upper_lip,lower_lip)
    
    return dist1,left_dist,right_dist,down_dist,ear

### Model()
The main code of ADAS.

In [7]:
def Model(draw=True,camera=0, OBJ_DETECTION=True, DISTRACTION=True, DROWSINESS=True, SMOKING=False,
          audio_alarm=True, EYE_AR_THRESH=0.2, TIME_closed_eyes=3,TIME_open_mouth=2, Nb_Yawn=3,
          Alarm_time_yawn=60, Time_Alarm_Loop=5, direction_cfg = 0):
    
    #Initialisations
    face_length = 400
    down_threshold = 30
    EYE_AR_CONSEC_FRAMES = 40
    YAWN_THRESH = 30
    YAWN_CONSEC_FRAMES = 40
    DIST_CONSEC_FRAMES = 30
    ALARM_Thresh = 30
    cpt_yawn_per_minute = 0 
    alarm_status = False
    alarm_status2 = False
    cpt = cpt2 = cpt3 = cpt4 = cpt5 = cpt6 = cpt7 = 0
    speed = 80 # km/h

    #OBJ DETECTION confguration
    configPath = 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
    weightsPath = 'frozen_inference_graph.pb'
    net = cv2.dnn_DetectionModel(weightsPath,configPath)
    net.setInputSize(300,300)
    net.setInputScale(1.0/127.5)
    net.setInputMean((127.5,127.5,127.5))
    net.setInputSwapRB(True)

    cap = cv2.VideoCapture(camera)
    pTime = 0
    while cv2.waitKey(1) != 27 :
        #vehicle speed simulation:
    #     if cv2.waitKey(1) == ord('a'):
    #         speed += 10
    #     if cv2.waitKey(1) == ord('d'):
    #         speed -= 10
    #     if speed> 100:
    #         speed = 0
    #     if speed<0:
    #         speed = 100
        #cv2.putText(display,str(round(speed))+'km/h',(200,100),cv2.FONT_HERSHEY_COMPLEX,.8,(0,255,0),2)
        success, img= cap.read()
        if not success:
            print("No image!")
            break
        display = img
        faces = []

        ########## OpenCV Object Detection ##########
        if OBJ_DETECTION:
            classIds, confs, bbox = net.detect(img,confThreshold=0.5)
            if len(classIds) != 0:
                for classId,confidence,box in zip(classIds.flatten(),confs.flatten(),bbox):
                    if classId in classes:
                        if draw:
                            cv2.rectangle(display,box,color=(0,255,0),thickness=2)
                            cv2.putText(display,classNames[classId].upper(),(box[0]+10,box[1]+30),cv2.FONT_HERSHEY_COMPLEX,.8,(0,255,0),2)
                        if audio_alarm:
                            cpt3 += 1
                            if cpt3 == 1:
                                mixer.Sound(FoodAudioFile).play()
                            if cpt3 >= ALARM_Thresh:
                                cpt3 = 0
                        cv2.putText(display,'FOOD DETECTED',(200,400),cv2.FONT_HERSHEY_COMPLEX,.8,(0,0,255),2)

                    if classId==77:
                        
                        cv2.rectangle(display,box,color=(0,255,0),thickness=2)
                        cv2.putText(display,'CELLPHONE DETECTED',(200,400),cv2.FONT_HERSHEY_COMPLEX,.8,(0,0,255),2)
                        if audio_alarm:
                            cpt4 += 1
                            if cpt4 == 1: 
                                mixer.Sound(PhoneUsageAudioFile).play()
                            if cpt4 >= ALARM_Thresh:
                                cpt4 = 0

        ########## Smoking Detection ##########
        if SMOKING:
            predicted=[]
            predicted.append(cv2.resize(img, (224,224)))
            pred=model.predict(np.array(predicted)/255)
            cv2.putText(display,str(round(pred[0][0],2)),(200,200),cv2.FONT_HERSHEY_COMPLEX,.8,(0,0,255),2)
            if pred[0][0]>0.5:
                cv2.putText(display,'Smoking DETECTED',(200,100),cv2.FONT_HERSHEY_COMPLEX,.8,(0,255,0),2)

                if audio_alarm:
                    cpt6 +=1
                    if cpt6 == 4:
                        mixer.Sound(SmokingAudioFile).play()
                    if cpt6 >= ALARM_Thresh:
                        cpt6 = 0

        ########## Mediapipe FaceMesh Detection ##########
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if DISTRACTION or DROWSINESS:
            results = faceMesh.process(imgRGB)
            if results.multi_face_landmarks:
                for faceLms in results.multi_face_landmarks:
                    if draw:
                        mpDraw.draw_landmarks(display,faceLms,mpFaceMesh.FACEMESH_CONTOURS,drawSpec,drawSpec)
                    face = []
                    for id, lm in enumerate(faceLms.landmark):
                        ih, iw, ic = img.shape
                        x,y,z = int(lm.x*iw), int(lm.y*ih), int(lm.z*ic)
                        # to show the landmarks ids
                        #cv2.putText(img, str(id), (x,y),cv2.FONT_HERSHEY_PLAIN,0.8, (0,255,0), 1)
                        face.append([x,y,z])

                    down_threshold = down_threshold / face_length
                    face_length = dist.euclidean(face[10],face[152])
                    down_threshold = down_threshold * face_length
                    
                    dist1,left_dist,right_dist,down_dist,ear= Calculations(face)
                    cv2.putText(img, str(round(DIST_CONSEC_FRAMES)), (150,160),cv2.FONT_HERSHEY_PLAIN,2, (0,0,255), 2)
                    direction= (right_dist-left_dist)/face_length


                    ########## Alarm conditions ##########
                    if DISTRACTION and speed!=0:
                        if direction<-0.5+direction_cfg or down_dist<=down_threshold or direction>0.5+direction_cfg:
                            cpt7 += 1
                            #cv2.putText(display, str(cpt7)+'/'+str(DIST_CONSEC_FRAMES), (150,250),cv2.FONT_HERSHEY_PLAIN,2, (0,0,255),3)
                            if cpt7 >= DIST_CONSEC_FRAMES:
                                mixer.Sound(DistratedAudioFile).play()
                                cpt7 = 0
                        else:
                            cpt7 = 0

                    if DROWSINESS and speed!=0:
                        if ear < EYE_AR_THRESH:
                            cpt += 1
                            cv2.putText(display, str(cpt), 
                                    (50,100),cv2.FONT_HERSHEY_PLAIN,
                                    3, (0,0,255),2)
                            if cpt >= EYE_AR_CONSEC_FRAMES:
                                cv2.putText(display, 'SLEEP ALERT!', (120,100),cv2.FONT_HERSHEY_PLAIN,3, (0,0,255),2)
                                cpt = 0 #
                                if alarm_status == False:
                                    alarm_status = True
                                    if audio_alarm:
                                        mixer.Sound(ClosedEyesAudioFile).play()
                                        alarm_status = False #

                        else:
                            cpt = 0
                            alarm_status = False

                        YAWN_THRESH = dist.euclidean(face[78],face[308])/1.5

                        if dist1 > YAWN_THRESH:
                            cpt2 += 1
                            if cpt2 == 1:
                                yawn = True
                            cv2.putText(display, str(cpt2), (50,150),cv2.FONT_HERSHEY_PLAIN, 3, (0,0,255),2)
                            if cpt2 >= YAWN_CONSEC_FRAMES:
                                cv2.putText(img, 'Yawn alert', (120,150),cv2.FONT_HERSHEY_PLAIN,3, (0,0,255),2)
                                if yawn == True:
                                    yawn = False
                                    cpt_yawn_per_minute += 1
                                    if cpt_yawn_per_minute == 1:
                                        tm = time.time()
                                if cpt_yawn_per_minute >=  Nb_Yawn:
                                    if time.time() - tm <= Alarm_time_yawn :
                                        if alarm_status2 == False:
                                            alarm_status2 = True
                                            if audio_alarm:
                                                mixer.Sound(TiredAudioFile).play()
                                    else:
                                        tm = time.time()
                                        cpt_yawn_per_minute = 1
                        else:
                            yawn = False
                            cpt2 = 0
                            alarm_status2 = False



                        cv2.putText(display, 'Yawn: '+str(round(dist1,2)), 
                                    (10,200),cv2.FONT_HERSHEY_PLAIN,
                                    1.5, (0,255,0),1)
                        cv2.putText(display, 'EAR: '+str(round(ear,2)), 
                                    (10,240),cv2.FONT_HERSHEY_PLAIN,
                                    1.5, (0,255,0),1)

                        #
                        cv2.putText(display, 'Yawn per minute : '+str(cpt_yawn_per_minute), 
                                    (10,280),cv2.FONT_HERSHEY_PLAIN,
                                    1.5, (0,255,0),1)
                        
                    faces.append(face)
    # Hot-keys short-cut
    #     if Help:
    #         cv2.putText(img,"'D': Detect drowsiness ",(20,340),cv2.FONT_HERSHEY_PLAIN, 1, (10,50,10), 2)
    #         cv2.putText(img,"'S': Detect distraction ",(20,380),cv2.FONT_HERSHEY_PLAIN, 1, (10,50,10), 2)
    #         cv2.putText(img,"'O': Detect objects ",(20,420),cv2.FONT_HERSHEY_PLAIN, 1, (10,50,10), 2)
    #         cv2.putText(img,"'H': Show this information ",(20,450),cv2.FONT_HERSHEY_PLAIN, 1, (10,50,10), 2)

    #     if cv2.waitKey(1) ==  ord('h'):
    #         if Help:
    #             Help = False
    #         else:
    #             Help = True
    #     if cv2.waitKey(1) ==  ord('d'):
    #         if DROWSINESS:
    #             DROWSINESS = False
    #         else:
    #             DROWSINESS = True
    #     if cv2.waitKey(1) ==  ord('s'):
    #         if DISTRACTION:
    #             DISTRACTION = False
    #         else:
    #             DISTRACTION = True
    #     if cv2.waitKey(1) ==  ord('o'):
    #         if OBJ_DETECTION:
    #             OBJ_DETECTION = False
    #         else:
    #             OBJ_DETECTION = True

        #calcul of fps
        cTime = time.time()
        fps = 1/(cTime-pTime)
        #readapt thresholds according to your fps
        DIST_CONSEC_FRAMES = fps * Time_thresh(speed,2,20,5)
        EYE_AR_CONSEC_FRAMES = fps * Time_thresh(speed,2,8,3)
        YAWN_CONSEC_FRAMES = fps * TIME_open_mouth
        ALARM_Thresh = fps * Time_Alarm_Loop
        pTime = cTime 
        cv2.putText(img,f'FPS: {int(fps)}',(20,70),cv2.FONT_HERSHEY_PLAIN, 3, (0,255,0), 3)
        cv2.imshow("ADAS", display)
    # to show vehicle velocity:
    #     if cv2.waitKey(1) == ord('a'):
    #         cv2.imwrite("image4.png",display)

    #     plt.xlabel("Speed (km/h)")
    #     plt.ylabel("Time threshold (s)")
    #     plt.grid('on')
    #     x=np.array(range(50))
    #     plt.plot(x,Time_thresh(x),'r')
    #     plt.plot(speed, Time_thresh(speed), 'bo')
    #     plt.savefig('ploy.jpg')
    #     cv2.imshow("plot",cv2.imread('ploy.jpg'))
    #     plt.clf()

    #release the camera and close the windows
    cap.release()
    cv2.destroyAllWindows()


# Parameters 

In [8]:
draw = True
First_cfg = False

OBJ_DETECTION = True # unable/disable Object detection
DISTRACTION = True # unable/disable distraction detection
DROWSINESS = True # unable/disable drowsiness detection
SMOKING = False # unable/disable smoking detection

audio_alarm = True # unable/disable audio alarms
EYE_AR_THRESH = 0.2 # Eye aspect ratio threshold
TIME_open_mouth = 2 # seconds of open mouth frames to detect a yawn
TIME_closed_eyes = 2 # seconds of closed eyes to detect sleeping
Nb_Yawn = 3 # Number of Yawnings per 'Alarm_time_yawn'
Alarm_time_yawn = 60 #Seconds in which yawn counter will be reset
Time_Alarm_Loop = 3   #Seconds 
camera = 1 #Camera id

# Main

In [12]:
mixer.init()
model = keras.models.load_model('smoking_model')
#face mesh params and drawing options
mpDraw = mp.solutions.drawing_utils
mpFaceMesh = mp.solutions.face_mesh
faceMesh = mpFaceMesh.FaceMesh(max_num_faces=1)
drawSpec = mpDraw.DrawingSpec(thickness=1,circle_radius=0)

if First_cfg:
    direction_cfg = FIRST_CONFIGURATION()
else: 
    direction_cfg = 0
Model(draw,camera, OBJ_DETECTION, DISTRACTION, DROWSINESS,SMOKING,audio_alarm, EYE_AR_THRESH,
      TIME_closed_eyes,TIME_open_mouth, Nb_Yawn,Alarm_time_yawn, Time_Alarm_Loop, direction_cfg)


NameError: name 'keras' is not defined